# Joint Intent Classification and Slot Filliing with Transformers

This notebook is based on the Deep Learning course from the Master Datascience Paris Saclay. Materials of the course can be found [here](https://github.com/m2dsupsdlclass/lectures-labs).

**Goal**
* Fine-tune a pretrained transformer-based neural network model to convert a user qeury expressed in English into a representation that is structured enough to be processed by an automated service.

Here is an example of interpretation computed by such a Natural Language Understanding system:
    
    >>> nlu('Book a table for two at Le Ritz for Friday night",
            tokenizer, joint_model, intent_names, slot_names)
    {
        'intent': 'BookRestaurant',
        'slots': {
            'party_size_number': 'two',
            'restaurant_name': 'Le Ritz',
            'timeRange': 'Friday night'
        }
    }
    
Intent classification is a simple classification problem. The trick is to treat the structured knowledge extraction part ("Slot Filling") as a token-level classification problem using BIO-annotations:

    >>> show_predictions('Book a table for two at Le Ritz for Friday night',
                         tokenizer, joint_model, intent_names, slot_names)
    ## Intent: BookRestaurant
    ## Slots:
      Book : O
         a : O
     table : O
       for : O
       two : B-party_size_number
        at : O
        Le : B-restaurant_name
         R : I-restaurant_name
     ##itz : I-restaurant_name
       for : O
    Friday : B-timeRange
     night : I-timeRange
     
We will show hhow to train a such "sequence classification" and "token classification" joint model on a voice command dataset published by snips.ai. This notebook is a partial reproduction of some of the results presented in this paper: BERT for Joint Intent Classification and Shot Filling, Qian Chen, Zhu Zhuo, Wen Wang [link](https://arxiv.org/abs/1902.10909).

In [ ]:
# Load packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from transformers import BertTokenizer, TFBertModel
from urllib.request import urlretrieve

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

## The data

We will use a speed command dataset collected, annotated and published by French startup snips.ai (bought in 2019 by Audio device manufacturer Sonos). The original dataset comes in [YAML format with inline markdown annotations](https://snips-nlu.readthedocs.io/en/latest/dataset.html). Instead, we will use a preprocessed variant with token level B-I-O annotations closer to the representation our model will predict. This variant of the snips dataset was prepared by [Su Zhu](https://github.com/sz128).

In [ ]:
# SNIPS_DATA_BASE_URL = (
#     "https://github.com/ogrisel/slot_filling_and_intent_detection_of_SLU/blob/"
#     "master/data/snips/"
# )
# for filename in ["train", "vocab.intent", "vocab.slot"]:
#     path = Path(filename)
#     if not path.exists():
#         print(f"Downloading {filename}...")
#         urlretrieve(SNIPS_DATA_BASE_URL + filename + "?raw=true", path)

Let's have a look at the first lines from the training set.

In [ ]:
import random


lines_train = Path('/kaggle/input/qqqqqqqqqqqqqqqqqqqqq/train').read_text('utf-8').strip().splitlines()
random.shuffle(lines_train)


In [ ]:
print(f'First line of training set: {lines_train[0]}.')

Some remarks:
* The class label for the voice command appears at the end of each line (after the "<=>" marker).
* Each word-level token is annotated with B-I-O labels using the "." separator.
* B-I-O stands for Beginning-Inside-Outside
* "Add:O" means that the token "Add" is "Outside" of any annotation span.
* "Don:B-entity_name" means that "Don" is the "Beginning" of an annotation of type "entity_name".
* "and:I-entity_name" means that "and" is "inside" the previously started annotation of type "entity_name".

Let's write a parsing function and test it on the first line.

In [ ]:
def parse_line(line):
    utterance_data, intent_label = line.split(" <=> ")
    items = utterance_data.split()
    words = [item.rsplit(':', 1)[0] for item in items]
    word_labels = [item.rsplit(':', 1)[1] for item in items]
    return {
        'intent_label': intent_label,
        'words': " ".join(words),
        'words_label': " ".join(word_labels),
        'length': len(words)
    }

In [ ]:
parse_line(lines_train[0])

This utterance is a voice command of type "AddToPlayist" with annotations:
* an entity-name: "Don and Sherri",
* a playlist: "Medidate to Sounds of Nature".

The goal of this project is to build a baseline Natural Understanding model to analyse such voice commands and predict:
* the intent of the speaker: the sentence level class label ("AddToPlaylist");
* extract the interesting slots (typed named entities) from the sentence by performing word level classification using the B-I-O tags as target classes. This second task is often referred to as "NER" (Named Entity Recognition) in the NLP litterature. Alternatively, this is also known as "slot filling" when we expect a fixed set of named entity per sentence of a given class.

The list of possible classes for the sentence level and the word level classification problems are given as:

In [ ]:
print(Path('/kaggle/input/vocab-intent-slot/vocab.intent').read_text('utf-8'))

In [ ]:
print(Path('/kaggle/input/qqqqqqqqqqqqqqqqqqqqq/vocab.slot').read_text('utf-8'))

"POI" stands for "Point of Interest". Let's parse all the lines and store the results in a Pandas dataframes.

In [ ]:
# for line in lines_train:
#     print(line)
#     print(parse_line(line))

parsed = [parse_line(line) for line in lines_train]
df_train = pd.DataFrame([p for p in parsed if p is not None])

In [ ]:
# Print some lines of the training set
df_train.head(5)

In [ ]:
# Count the number of lines by intent label
df_train.intent_label.value_counts()

In [ ]:
# Histogram of sentence lengths
df_train.hist('length', bins=30)

In [ ]:
# Get validation and test set
# lines_validation = Path('valid').read_text('utf-8').strip().splitlines()
# lines_test = Path('test').read_text('utf-8').strip().splitlines()

# df_validation = pd.DataFrame([parse_line(line) for line in lines_validation])
# df_test = pd.DataFrame([parse_line(line) for line in lines_test])

## Intent classification (sentence level)

Let's ignore the slot filling task for now and let's try to build a sentence level classifier by fine-tuning a pre-trained Transformer-based model using the `huggingface/transformers` package that provides both Tensorflow/Keras and Pytorch APIs.

### The BERT tokenizer

First let's load a pre-trained tokenizer and test it on a test sentence from the training set.

In [ ]:
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
first_sentence = df_train.iloc[0]['words']
print(first_sentence)

In [ ]:
tokenizer.tokenize(first_sentence)

Notive that BERT uses subword tokens. So the length of the tokenized sentence is likely to be larger than the number of words in the sentence. It is particularly interesting to use subword tokenization sentence for general purpose language models such as BERT because it should be possible to generalize the model and then to fine-tuned it to be a specialized one.

Each token string is mapped to a unique integer id that makes it fast to lookup the right column in the input layer token embedding.

In [ ]:
# Encode sentence to id
tokenizer.encode(first_sentence)

In [ ]:
# Do the inverse operation
tokenizer.decode(tokenizer.encode(first_sentence))

Remarks:
* The first token `[CLS]` is used by the pre-training task for sequence classification.
* The last token `[SEP]` is a separator for the pre-training task that classifies if a pair of sentences are consecutive in a corpus or not (next sentence prediction).
* Here, we want to use BERT to compute a representation of a single voice command at a time.
* We could reuse the representation of the `[CLS]` token for sequence classification.
* Alternatively, we can pool the representations of all the tokens of the voice command (*e.g.* global average) and use that as the input of the final sequence classification layer.

In [ ]:
train_sequence_lengths = [len(tokenizer.encode(text))
                          for text in df_train['words']]
plt.hist(train_sequence_lengths, bins=30)
plt.title(f'Max sequence length: {max(train_sequence_lengths)}')
plt.xlabel('Length')
plt.ylabel('Count')
plt.show()

To perform transfer learning, we will need to work with padded sequences. So, they all have the same sizes. The above histograms, shows that after tokenization, $43$ tokens are enough to represent all the voice commands in the training set.

The mapping can be introspected in the `tokenizer.vocab` attribute.

In [ ]:
print(f'Vocabulary size: {tokenizer.vocab_size} words.')

In [ ]:
# Get the items in BERT
bert_vocab_items = list(tokenizer.vocab.items())

In [ ]:
# Print some examples of items
bert_vocab_items[250:260]

Remarks:
* $30000$ is a reasonable vocabulary size and is small enough to be used in a softmax output layer.
* It can represent multi-lingual sentences, including non-western alphabets.
* Subword tokenization makes it possible to deal with typos and morphological variations with a small vocabulary size and without any language-specific preprocessing.
* Subword tokenization makes it unlikely to use the `[UNK]` special token as rare words can often be represented as a sequence of frequent enough short subwords in a meaningful way.

### Encoding the dataset with the tokenizer

Let's now encode the full train/validation and test sets with our tokenizer to get a padded integer numpy arrays.

In [ ]:
def encode_dataset(tokenizer, text_sequences, max_length):
    token_ids = np.zeros(shape=(len(text_sequences), max_length),
                         dtype=np.int32)
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded
    attention_masks = (token_ids != 0).astype(np.int32)
    
    return {'input_ids': token_ids, 'attention_masks': attention_masks}

In [ ]:
encoded_train = encode_dataset(tokenizer, df_train['words'], 53)

In [ ]:
encoded_train['input_ids']

In [ ]:
encoded_train['attention_masks']

### Encoding the sequence classification targets

To do so, we build a simple mapping from the auxiliary files.

In [ ]:
intent_names = Path('/kaggle/input/vocab-intent-slot/vocab.intent').read_text('utf-8').split()
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))

In [ ]:
intent_map

In [ ]:
intent_train = df_train['intent_label'].map(intent_map).values
# intent_validation = df_validation['intent_label'].map(intent_map).values
# intent_test = df_test['intent_label'].map(intent_map).values

### Loading and feeding a pretrained BERT model

Let's load a pretrained BERT model using the [huggingface transformers](https://github.com/huggingface/transformers) package.

In [ ]:
base_bert_model = TFBertModel.from_pretrained('bert-base-cased')
base_bert_model.summary()

In [ ]:
# outputs = base_bert_model(encoded_validation)

The first output of the BERT model is a tensor with shape: `(batch_size, seq_len, output_dim)` which computes features for each token in the input sequence.

In [ ]:
# print(f'Shape of the first output of the BERT model: {outputs[0].shape}.')

The second output of the BERT model is a tensor with shape `(batch_size, output_dim)` which is the vector representation of the special token `[CLS]`. This vector is typically used as a pooled representation for the sequence as a whole. This will be used as the features of our latent classifier.

In [ ]:
# print(f'Shape of the second output of the BERT model: {outputs[1].shape}.')

Let's build an train a sequece classification model using to predict the intent class. We will use the `self.bert` pretrained model in the `call` method and only consider the pooled features (ignore the token-wise features for now).

In [ ]:
# Define IntentClassification model
class IntentClassificationModel(tf.keras.Model):
    def __init__(self, intent_num_labels=None,
                 model_name='bert-base-cased',
                 dropout_prob=0.1):
        super().__init__(name='joint_intent_slot')
        # Let's preload the pretrained model BERT in the constructor
        # of our classifier model.
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        
        # Define a (Dense) classification layer to compute for each
        # sequence in a batch of samples. The number of output classes
        # is given by the intent_num_labels parameter.
        # Use the default linear activation (no softmax) to compute
        # logits. The softmax normalization will be computed in the
        # loss function instead of the model itself.
        self.intent_classifier = Dense(intent_num_labels)
        
    def call(self, inputs, **kwargs):
        # Use the pretrained model to extract features from our
        # encoded inputs.
        sequence_output, pooled_output = self.bert(inputs, **kwargs)
        
        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim) and gives a "pooled" representation
        # for the full sequence from the hidden state that corresponds
        # to the "[CLS]" token.
        pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
        
        # Use the classifier layer to compute the logits from the
        # pooled features.
        intent_logits = self.intent_classifier(pooled_output)
        return intent_logits

The classification model outputs logits instead of probabilities. The final softmax normalization layer is implicit, that is, included in the loss function instead of the model directly. We need to configure the loss function `SparseCategoricalCrossentropy(from_logits=True)` accordingly.

In [ ]:
# Build the model
intent_model = IntentClassificationModel(intent_num_labels=len(intent_map))

intent_model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08),
                     loss=SparseCategoricalCrossentropy(from_logits=True),
                     metrics=[SparseCategoricalAccuracy('accuracy')])

In [ ]:
# Train the model
history = intent_model.fit(encoded_train, intent_train,
                           epochs=4, batch_size=32)

In [ ]:
def classify(text, tokenizerzer, model, intent_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :] # Batch size = 1
    class_id = model(inputs).numpy().argmax(axis=1)[0]
    return intent_names[class_id]

In [ ]:
# Example of classification
classify('Swap my 30 UNI to SUSHI on aurora',
         tokenizer, intent_model, intent_names)

## Join intent classification and slot filling

Let's now refine our natural language understanding system by trying to retrieve the important structured elements of each voice command. To do so, we will perform word level (or token level) classification of the BIO labels. Since we have word level tags but BERT uses a wordpiece tokenizer, we need to align the BIO labels with the BERT tokens. Let's load the list of possible word token labels and augment it with an additional padding label to be able to ignore special tokens.

In [ ]:
slot_names = ["[PAD]"]
slot_names += Path('/kaggle/input/qqqqqqqqqqqqqqqqqqqqq/vocab.slot').read_text('utf-8').strip().splitlines()

slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

The following function generates token-aligned integer labels from the BIO word-level annotations. In particular, if a specific word is too long to be represented as a single token, we expand its label for all the tokens of that word while taking care of using "B-" labels only for the first token and then use "I-" for the matching slot type for subsequent tokens of the same word.

In [ ]:
def encode_token_labels(text_sequences, slot_names, tokenizer, slot_map, max_length):
    encoded = np.zeros(shape=(len(text_sequences), max_length), dtype=np.int32)
    for i, (text_sequence, word_labels) in enumerate(
            zip(text_sequences, slot_names)):
        encoded_labels = []
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded

In [ ]:
slot_train = encode_token_labels(df_train['words'], df_train['words_label'], tokenizer, slot_map, 53)
# slot_validation = encode_token_labels(df_validation['words'], df_validation['words_label'], tokenizer, slot_map, 45)
# slot_test = encode_token_labels(df_test['words'], df_test['words_label'], tokenizer, slot_map, 45)

Note that the special tokens such as "[PAD]" and "[SEP]" and all padded positions receive a $0$ label.

In [ ]:
# Define JointIntentAndSlotFilling model
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                 model_name="bert-base-cased", dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels,
                                       name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")

    def call(self, inputs, **kwargs):
        sequence_output, pooled_output = self.bert(inputs, **kwargs)

        # The first output of the main BERT layer has shape:
        # (batch_size, max_length, output_dim)
        sequence_output = self.dropout(sequence_output,
                                       training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim)
        # and gives a "pooled" representation for the full sequence from the
        # hidden state that corresponds to the "[CLS]" token.
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits

In [ ]:
joint_model = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

# Define one classification loss for each output:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]
metrics = [SparseCategoricalAccuracy('accuracy')]
joint_model.compile(optimizer=opt, loss=losses , metrics=metrics)

In [ ]:
history = joint_model.fit(
    encoded_train, (slot_train, intent_train),
    epochs=4, batch_size=32)

The following function uses our trained model to make prediction on a single text sequence and display both the sequence-wise and the token-wise class labels.

In [ ]:
def show_predictions(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    print("## Intent:", intent_names[intent_id])
    print("## Slots:")
    for token, slot_id in zip(tokenizer.tokenize(text), slot_ids):
        print(f"{token:>10} : {slot_names[slot_id]}")

In [ ]:
# Example of classification
show_predictions('Swap my 30 UNI to SUSHI on aurora',
                 tokenizer, joint_model, intent_names, slot_names)

## Decoding predictions into structured knowledge

For completeness, here a minimal functional to naively decode the predicted BIO slot ids and convert it into a structured representation for the detected slots as a Python dictionaries.

In [ ]:
def decode_predictions(text, tokenizer, intent_names, slot_names,
                       intent_id, slot_ids):
    info = {"intent": intent_names[intent_id]}
    collected_slots = {}
    active_slot_words = []
    active_slot_name = None
    for word in text.split():
        tokens = tokenizer.tokenize(word)
        current_word_slot_ids = slot_ids[:len(tokens)]
        slot_ids = slot_ids[len(tokens):]
        current_word_slot_name = slot_names[current_word_slot_ids[0]]
        if current_word_slot_name == "O":
            if active_slot_name:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = []
                active_slot_name = None
        else:
            # Naive BIO: handling: treat B- and I- the same...
            new_slot_name = current_word_slot_name[2:]
            if active_slot_name is None:
                active_slot_words.append(word)
                active_slot_name = new_slot_name
            elif new_slot_name == active_slot_name:
                active_slot_words.append(word)
            else:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = [word]
                active_slot_name = new_slot_name
    if active_slot_name:
        collected_slots[active_slot_name] = " ".join(active_slot_words)
    info["slots"] = collected_slots
    return info

In [ ]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    return decode_predictions(text, tokenizer, intent_names, slot_names,
                              intent_id, slot_ids)

In [ ]:
nlu('Swap my 30 UNI to SUSHI on aurora',
                 tokenizer, joint_model, intent_names, slot_names)

In [ ]:
nlu('swap my 20 eth to dai on ethereum',
    tokenizer, joint_model, intent_names, slot_names)

In [ ]:
# joint_model.save('v1_model_202_params')
joint_model.save_weights('latest/weights')

In [ ]:
[len(intent_map), len(slot_map)]

In [ ]:


load_model1 = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

# Define one classification loss for each output:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]
metrics = [SparseCategoricalAccuracy('accuracy')]
load_model1.compile(optimizer=opt, loss=losses , metrics=metrics)

In [ ]:
# load_model1 = joint_model.compile(optimizer=opt, loss=losses , metrics=metrics)
load_model1.load_weights('latest/weights')

nlu('swap my 20 eth to dai on ethereum',
    tokenizer, load_model1, intent_names, slot_names)

In [ ]:
!zip -r file.zip /kaggle/working/latest


In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')


In [ ]:
# tf.__version__


In [ ]:
# %pip install flask-ngrok

In [ ]:
# !wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz


In [ ]:
# !tar xvzf ./ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin
# !ngrok config add-authtoken 2ZI9uXWhLuz1WZQvTgLnXsQsLkZ_7P4qfg9ezSbuqfVip3z89


In [ ]:
# !npm install -g localtunnel

In [ ]:
# !ngrok config add-authtoken 2ZI9uXWhLuz1WZQvTgLnXsQsLkZ_7P4qfg9ezSbuqfVip3z89

In [ ]:
# %pip install ngrok

In [ ]:
# !pip install pyngrok==4.1.1
# !ngrok authtoken '2ZI9uXWhLuz1WZQvTgLnXsQsLkZ_7P4qfg9ezSbuqfVip3z89'

In [ ]:
# from flask import Flask, jsonify, request
# from flask_ngrok import run_with_ngrok


# app = Flask(__name__)
# run_with_ngrok(app)
# # listener =  ngrok.forward(4040 , authtoken="2ZI9uXWhLuz1WZQvTgLnXsQsLkZ_7P4qfg9ezSbuqfVip3z89")

# @app.route('/predict' , methods=["POST"])
# def hello():
#     if request.method != "POST":return make_response(jsonify({"message": "Method not allowed"}), 405)
#     data = request.get_json()
#     userText = data["Text"]
#     return nlu(
#         userText,
#         tokenizer,
#         load_model1,
#         intent_names,
#         slot_names,
#     )

# @app.route('/test')
# def hello():
#     return "test"
# # print(f"Ingress established at {listener}")

# if __name__ == '__main__':
#     app.run()
    


## Limitations

### Language

BERT is pretrained primarily on English content. It can therefore only extract meaningful features on text written in English.

Note that there exists alternative pretrained model that use a mix of different languages (e.g. [XLM](https://github.com/facebookresearch/XLM/) and others that have been trained on other languages. For instance [CamemBERT](https://camembert-model.fr/) is pretrained on French text. Both kinds of models are available in the transformers package:

https://github.com/huggingface/transformers#model-architectures

The public snips.ai dataset used for fine-tuning is English only. To build a model for another language we would need to collect and annotate a similar corpus with tens of thousands of diverse, representative samples.

### Biases embedded in the pre-trained model

The original data used to pre-trained BERT was collected from the Internet and contains all kinds of data, including offensive and hateful speech.

While using BERT for or voice command understanding system is quite unlikely to be significantly impacted by those biases, it could be a serious problem for other kinds of applications such as Machine Translation for instance.

It is therefore strongly recommended to spend time auditing the biases that are embedded in such pre-trained models before deciding to deploy system that derive from them.

### Computational ressources

The original BERT model has many parameters which uses a lot of memory and can be prohibitive to deploy on small devices such as mobile phones. It is also very computationally intensive and typically requires powerful GPUs or TPUs to process text data at a reasonable speed (both for training and at inference time).

Designing alternative architectures with fewer parameters or more efficient training and inference procedures is still a very active area of research.

Depending of on the problems, it might be the case that simpler architectures based on convolutional neural networks and LSTMs might offer a better speed / accuracy trade-off.